In [ ]:
using Base.Test, JuLIPMaterials, JuLIP, ForwardDiff, Plots
Plots.pyplot()

MST = JuLIPMaterials
CB = MST.CauchyBorn
CLE = MST.CLE
FD = ForwardDiff

In [ ]:
# setup a Cauchy-Born model for bulk Fe
atu = bulk(:Fe)
r0 = rnn(:Fe)
calc = LennardJones(σ = r0) * C2Shift(2.7*r0)
set_calculator!(atu, calc)
set_constraint!(atu, VariableCell(atu))
println("Constructing Wcb.")
# normalise CB potential w.r.t. # atoms instead of volume
W = CB.Wcb(atu, calc, normalise = :atoms)
;

In [ ]:
# show a bit how it behaves
@show energy(atu) - W(eye(3))
@show CB.grad(W, eye(3)) - (-virial(atu))
;

In [ ]:
# compute atomistic and Cauchy-Born forces on 
# a large cluster 

R = 62.1
at = cluster(:Fe, R)
X0 = positions(at)    # reference positions
x0 = X0[1]            # center of cluster 

# a nice, smooth displacement with 1/r decay 
p = -1
const A = eye(3) #+rand(3,3)
const B = eye(3) #+rand(3,3)
y = x -> x + (2 + sum(dot(x-x0, A*(x-x0))))^((p-1)/2) * B * (x - x0)
∇y = x -> FD.jacobian(y, x)

# setup the atomistic domain and model 
set_calculator!(at, calc)
set_positions!(at, y.(X0))

# distance from center 
r = 1 + norm.([x - x0 for x in X0])
# atomistic forces 
Fat = forces(at)
# cauchy-born forces 
Fcb = [ CB.div_grad(W, ∇y, x)  for x in X0 ]
# force error 
Ferr = norm.(Fat - Fcb)
;

In [ ]:
Iin = find(r .< 45)
t = [extrema(r[Iin])...]
P = plot(r[Iin], 1e-15+norm.(Fat[Iin]), lw=0, m=:+, ms=2, label = "|f_at|",
         xaxis = (:log, [1.0, 1.2*R]), yaxis = (:log, [1e-3, 1e2]) )
plot!(P, r[Iin], 1e-15+Ferr[Iin], lw = 0, m=:x, ms=2, label="|f_at - f_cb|")
plot!(P, t, 4_000*t.^(p-2), label = "r^{p-2}")
plot!(P, t, 3_000*t.^(p-3), label = "r^{p-3}")
plot!(P, t, 20_000*t.^(p-4), label = "r^{p-4}")
display(P)
# STRANGE - we get one power better than expected???? A symmetry thing again?
;